**1. Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:**

* InvoiceNo — номер транзакции
* StockCode — код товара
* Description — описание товара
* Quantity — количество единиц товара, добавленных в заказ
* InvoiceDate — дата транзакции 
* UnitPrice — цена за единицу товара
* CustomerID — id клиента
* Country — страна, где проживает клиент

**В данной задаче ссылка для считывания данных лежит в переменной path_to_file.
Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.**

In [5]:
import pandas as pd
path_to_file = 'data.csv'
retail = pd.read_csv(path_to_file, 
                 encoding='ISO-8859-1', sep=',')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
retail_columns = retail.columns
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

**2. Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.**

Вам понадобится len(название_колонки) и drop_duplicates()

In [13]:
len(retail)

541909

In [22]:
retail = retail.drop_duplicates()
len(retail)

536641

**3. Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled).**

**Сколько всего транзакций отменили пользователи? Каждый отдельный тип продукта считайте за отдельную транзакцию - уникальные номера считать не надо.**

Прежде чем начать, попробуйте метод retail.InvoiceNo.str[0]

Далее используйте метод .unique()

И сделайте выбоку по retail.loc[retail.InvoiceNo.str[0] == 'C'].shape

In [24]:
retail.InvoiceNo.str[0]

0         5
1         5
2         5
3         5
4         5
         ..
541904    5
541905    5
541906    5
541907    5
541908    5
Name: InvoiceNo, Length: 536641, dtype: object

In [25]:
retail.InvoiceNo.str[0].unique()

array(['5', 'C', 'A'], dtype=object)

In [26]:
 retail.loc[retail.InvoiceNo.str[0] == 'C'].shape

(9251, 8)

In [27]:
retail = retail.loc[retail.Quantity > 0]
retail.shape

(526054, 8)

**5.Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).**

Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

**Подсказка:**
1. Начните с выборки retail.Country == 'Germany', группировки по 'CustomerID' и агрегации total_invoices=('InvoiceNo', 'nunique')
2. Далее из полученой выборки из предыдущего шага выподните метод: название_выборки.столбец.quantile(0.8)
3. Запишите полученные id пользователей в germany_top

In [31]:
invoice_dist = retail.query('Country == "Germany"')\
                     .groupby('CustomerID', as_index=False)\
                     .agg(total_invoices=('InvoiceNo', 'nunique'))
invoice_dist.head()

,CustomerID,total_invoices
0,12426.0,1
1,12427.0,3
2,12468.0,2
3,12471.0,30
4,12472.0,7


In [29]:
N = invoice_dist.total_invoices.quantile(0.8)
N

np.float64(7.0)

In [30]:
germany_top = invoice_dist.loc[invoice_dist.total_invoices > N]['CustomerID']
germany_top

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

**6. Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.**

Датафрейм записан в retail, нужные id пользователей – в germany_top.

**Подсказка**
Попробуйте использовать top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]

In [33]:
top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


**7. Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?** 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [71]:
stock_code_germany = top_retail_germany.groupby('StockCode')\
                                       .size()\
                                       .sort_values(ascending=False)
stock_code_germany.head()

StockCode
POST     213
22326     52
22328     38
22423     34
20719     30
dtype: int64

**8. Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.**

Отфильтрованные данные записаны в retail.

**Подсказка**
retail['новый_столбец'] = retail.старый_столбец_1 * retail.старый_столбец_2

In [72]:
retail['Revenue'] = retail.UnitPrice * retail.Quantity
retail['Revenue']

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
541904    10.20
541905    12.60
541906    16.60
541907    16.60
541908    14.85
Name: Revenue, Length: 526054, dtype: float64